## Earthquakes Web Scraping
### Countries Collected

Nicaragua: https://ineter.gob.ni/articulos/areas-tecnicas/geofisica/monitoreo-de-sismos-en-tiempo-real.html

USA: http://folkworm.ceri.memphis.edu/REQ/html/event_list.html

Spain: https://www.ign.es/web/ign/portal/ultimos-terremotos/-/ultimos-terremotos/get10dias?_IGNGFSSismoSismicidadReciente_WAR_IGNGFSSismoSismicidadRecienteportlet_formDate=1706589250669&_IGNGFSSismoSismicidadReciente_WAR_IGNGFSSismoSismicidadRecienteportlet_dias=10

FRANCE: https://www.notre-planete.info/terre/risques_naturels/seismes/seismes-France-carte.php

In [1]:
#Importing libraries
from datetime import date
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import calendar
import pymysql
import re

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0"}


### MySQL connection

In [2]:
# Connect to the MySQL database
connection = pymysql.connect(
    host='127.0.0.1',     # Hostname or IP address of the MySQL server
    user='root', # MySQL username
    password='',     # MySQL password
    database='sismos_registros_nicaragua',     # Name of the MySQL database
)

### Table Creation and Connection

In [3]:
table_name = "registro_sismos"
create_table_query = f'''
CREATE TABLE IF NOT EXISTS {table_name} (
    Date DATETIME,
    Time TIME,
    Latitude DECIMAL(10, 6),
    Longitude DECIMAL(10, 6),
    Depth DECIMAL(10, 2),
    Magnitude DECIMAL(5, 2),
    Closest_City VARCHAR(100),
    Collected VARCHAR(45)
);
'''.format(table_name)

with connection.cursor() as cursor:
    cursor.execute(create_table_query)
connection.commit()

In [ ]:
## Getting Last Date recorded for US, NI and ES for web scraping
sql_ni = 'SELECT * FROM sismos_registros_nicaragua.registro_sismos WHERE Collected = "Nicaragua";'
sql_us = 'SELECT * FROM sismos_registros_nicaragua.registro_sismos WHERE Collected = "US";'
sql_es = 'SELECT * FROM sismos_registros_nicaragua.registro_sismos WHERE Collected = "Spain";'
sql_fr = 'SELECT * FROM sismos_registros_nicaragua.registro_sismos WHERE Collected = "France";'

last_date_ni = pd.read_sql(sql_ni, connection)['Date'].max()
last_date_us = pd.read_sql(sql_us, connection)['Date'].max()
last_date_es = pd.read_sql(sql_es, connection)['Date'].max()
last_date_fr = pd.read_sql(sql_fr, connection)['Date'].max()

## Nicaragua Database

In [ ]:
## Connect to Data
URL = "https://webserver2.ineter.gob.ni/geofisica/sis/events/sismos.php"

page = requests.get(URL, headers = headers,verify=False)

Soup1 = BeautifulSoup(page.content, "html.parser")

Soup2 = BeautifulSoup(Soup1.prettify(),"html.parser")

row = []
for fila in Soup2.find_all("pre"):
    row.append(fila.text)

In [7]:
## Creating lists for the pandas dataframe

date = []
time = []
latitude = []
longitude = []
depth = []
magnitude = []
region = []

for data in row:
    data = data.split()
    date.append(data[0])
    time.append(data[1])
    latitude.append(data[2])
    longitude.append(data[3])
    depth.append(data[4])
    magnitude.append(data[5])
    region.append(" ".join(data[7:]))
    

In [8]:
## Creating the dataframe and adding the country where it was registered
eq_record_ni = pd.DataFrame({"Date":date,"Time":time,"Latitude":latitude,
                        "Longitude":longitude, "Depth":depth,
                        "Magnitude":magnitude, "Region":region})

eq_record_ni["Date"] = pd.to_datetime(eq_record_ni.Date,format ="%y/%m/%d")
eq_record_ni["Collected"] = "Nicaragua"

In [9]:
## Filtering from last date saved in MySQL
eq_record_ni = eq_record_ni[eq_record_ni["Date"]>last_date_ni]

In [ ]:
eq_record_ni

## USA Database

In [ ]:
## Connect to Data
URLCentralUSA = "http://folkworm.ceri.memphis.edu/REQ/html/event_list.html"

page2 = requests.get(URLCentralUSA, headers = headers,verify=False)

Soup1_2 = BeautifulSoup(page2.content, "html.parser")

Soup2_2 = BeautifulSoup(Soup1_2.prettify(),"html.parser")


In [12]:
row = []
for fila in Soup2_2.find_all("td"):
    row.append(fila.text)

In [13]:
# Cleaning the data
clean_row = []
for i in row:
    i = i.strip()
    clean_row.append(i)

cleaning1 = [x for x in clean_row if len(x) == 10] 
    
for i in clean_row:
    if i in cleaning1:
        clean_row.remove(i)
    else:
        continue
        
titles = clean_row[1:7]
data_dirty = clean_row [7:]

In [14]:
## Creating lists for the pandas dataframe
DateTime_US = []
Latitude_US = []
Longitude_US = []
Depth_US = []
Magnitude_US = []
Region_US = []

n = 0

while n < (len(data_dirty)/6 + 1):
    Magnitude_US.append(data_dirty[n])
    DateTime_US.append(data_dirty[n+1])
    Latitude_US.append(data_dirty[n+2])
    Longitude_US.append(data_dirty[n+3])
    Depth_US.append(data_dirty[n+4])
    Region_US.append(data_dirty[n+5])
    n+=6

In [15]:
## Creating the dataframe and adding the country where it was registered
Dict_US = {"Magnitude":Magnitude_US, "Datetime":DateTime_US, "Latitude": Latitude_US,
           "Longitude":Longitude_US,"Depth":Depth_US, "Region": Region_US} 
eq_record_us = pd.DataFrame(Dict_US)

In [16]:
eq_record_us[['Day Name','Month','Day','Time','Year']] = eq_record_us['Datetime'].str.split(' ',expand=True)
eq_record_us = eq_record_us.drop(['Datetime', 'Day Name'], axis = 1)


In [17]:
months = {month: index for index, month in enumerate(calendar.month_abbr) if month}

eq_record_us = eq_record_us.replace({'Month':months})

In [18]:
eq_record_us = eq_record_us.astype({"Magnitude":"float64", "Latitude": "float64", "Longitude":"float64",
                                    "Depth":"float64", "Region": "object", "Month":"int64", "Day":"int64",
                                   "Time":"object", "Year":"int64"})

In [19]:
eq_record_us['Date'] = pd.to_datetime(eq_record_us[['Month','Day','Year']])
eq_record_us = eq_record_us.drop(['Month','Day','Year'], axis = 1)
eq_record_us['Collected'] = "US"
eq_record_us = eq_record_us[eq_record_ni.columns.to_list()]

In [20]:
eq_record_us = eq_record_us[eq_record_us["Date"]>last_date_us]

In [ ]:
eq_record_us

## Spain Database

In [22]:
## Connect to Data
URLSpain = "https://www.ign.es/web/ign/portal/ultimos-terremotos"

page3 = requests.get(URLSpain, headers = headers,verify=False)

Soup1_3 = BeautifulSoup(page3.content, "html.parser")

Soup2_3 = BeautifulSoup(Soup1_3.prettify(),"html.parser")

c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ign.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
row = []
for fila in Soup2_3.find_all("td"):
    row.append(fila.text)

In [24]:
clean_row = []
for i in row:
    i = i.strip()
    clean_row.append(i)

In [25]:
Date_ES = []
Time_ES = []
Latitude_ES = []
Longitude_ES = []
Depth_ES = []
Magnitude_ES = []
Region_ES = []

n = 1

while n < len(clean_row):
    Date_ES.append(clean_row[n])
    Time_ES.append(clean_row[n+1])
    Latitude_ES.append(clean_row[n+3])
    Longitude_ES.append(clean_row[n+4])
    Depth_ES.append(clean_row[n+5])
    Magnitude_ES.append(clean_row[n+6])
    Region_ES.append(clean_row[n+9])
    n+=12

In [26]:
Dict_ES = {"Magnitude":Magnitude_ES, "Date":Date_ES, "Time": Time_ES, "Latitude": Latitude_ES,
           "Longitude":Longitude_ES,"Depth":Depth_ES, "Region": Region_ES} 
eq_record_es = pd.DataFrame(Dict_ES)

In [27]:
eq_record_es["Collected"] = "Spain"
eq_record_es = eq_record_es[eq_record_ni.columns.to_list()]
eq_record_es["Date"] = pd.to_datetime(eq_record_es["Date"],dayfirst = True)

In [28]:
eq_record_es = eq_record_es[eq_record_es["Date"]>last_date_es]

In [29]:
eq_record_es

,Date,Time,Latitude,Longitude,Depth,Magnitude,Region,Collected
0,2024-04-06,19:16:36,38.2834,-0.8485,7.0,2.3,S EL FONDÓ DE LES NEUS/HONDÓN,Spain
1,2024-04-06,19:02:14,36.6088,-2.5524,13.0,1.8,ALBORÁN NORTE,Spain
2,2024-04-06,12:44:31,41.0459,-7.0684,0.0,1.8,SE VILA NOVA DE FOZ COA.POR,Spain
3,2024-04-06,07:57:49,37.1708,-3.3509,10.0,1.5,E GÜÉJAR SIERRA.GR,Spain
4,2024-04-06,01:02:28,28.0787,-16.1480,5.0,1.7,ATLÁNTICO-CANARIAS,Spain
...,...,...,...,...,...,...,...,...
57,2024-04-02,03:24:44,36.6334,-3.0300,16.0,2.0,ALBORÁN NORTE,Spain
58,2024-04-02,03:16:12,36.7875,-10.6935,7.0,2.7,SW CABO DE SAN VICENTE,Spain
59,2024-04-02,01:23:33,28.0057,-16.2008,5.0,1.8,ATLÁNTICO-CANARIAS,Spain
60,2024-04-02,00:35:40,35.4054,-3.7275,3.0,2.3,ALBORÁN SUR,Spain


## France Database

In [ ]:
## Connect to Data
URLFrance = "https://www.notre-planete.info/terre/risques_naturels/seismes/seismes-France-carte.php"
page4 = requests.get(URLFrance, headers = headers,verify=False)

Soup1_4 = BeautifulSoup(page4.content, "html.parser")

Soup2_4 = BeautifulSoup(Soup1_4.prettify(),"html.parser")

In [31]:
row = []
for fila in Soup2_4.find_all("td"):
    row.append(fila.text)

In [32]:
clean_row = []
for i in row:
    i = i.strip()
    clean_row.append(i)

In [33]:
Region_FR = []
for i in clean_row:
    try:
        Region_FR.append(i.split("proche de ")[1])
    except:
        continue

In [34]:
urls = re.findall(r'(https?://[^\s]+)', str(Soup2_4))

In [35]:
cleaned_urls = []
for i in urls:
    if i.startswith('https://renass.unistra.fr/fr/evenements/fr'):
        cleaned_urls.append(i[:len(i)-1])
    else:
        continue

In [36]:
## This is function extracts the data from every link in the main page from the cell above

def Record_Extractor(url):
    page= requests.get(url, headers = headers,verify=False)

    Soup= BeautifulSoup(page.content, "html.parser")

    Soup2= BeautifulSoup(Soup.prettify(),"html.parser")

    row = []
    for fila in Soup2.find_all("td"):
        row.append(fila.text)

    clean_row = []
    for i in row:
        i = i.strip()
        clean_row.append(i)

    Magnitude = repr(clean_row[3]).split("\\")[0][1:]
    Datetime = clean_row[7]
    Latitude = repr(clean_row[9]).split("\\")[0][1:]
    Longitude = repr(clean_row[11]).split("\\")[0][1:]
    Depth = repr(clean_row[13]).split("\\")[0][1:]
    
    data = [Datetime, Latitude, Longitude, Depth, Magnitude]

    return data

In [ ]:
## Connect to Data

Datetime_FR = []
Latitude_FR = []
Longitude_FR = []
Depth_FR = []
Magnitude_FR = []

for i in cleaned_urls:
    data = Record_Extractor(i)
    Datetime_FR.append(data[0])
    Latitude_FR.append(data[1])
    Longitude_FR.append(data[2])
    Depth_FR.append(data[3])
    Magnitude_FR.append(data[4])

In [39]:
## Creating the dataframe

Dict_FR = {"Magnitude":Magnitude_FR, "Date":Datetime_FR, "Latitude": Latitude_FR,
           "Longitude":Longitude_FR,"Depth":Depth_FR, "Region": Region_FR} 
eq_record_FR = pd.DataFrame(Dict_FR)

In [40]:
eq_record_FR[['Date', 'Time']] = eq_record_FR['Date'].str.split(' ', expand=True)

In [41]:
eq_record_FR['Date'] = pd.to_datetime(eq_record_FR['Date'])
eq_record_FR['Collected'] = 'France'

In [ ]:
eq_record_FR

## Union and load process to SQL

In [43]:
eq_global = pd.concat([eq_record_ni,eq_record_us,eq_record_es,eq_record_FR]).reset_index(drop=True)

In [ ]:
eq_global

In [45]:
df = eq_global
df_columns = list(df)
columns_str = ', '.join(df_columns)
values_placeholder = ', '.join(['%s'] * len(df_columns))

with connection.cursor() as cursor:
    insert_query = '''
        INSERT INTO {} ({}) VALUES ({})
    '''.format(table_name, columns_str, values_placeholder)
    cursor.executemany(insert_query, df.values.tolist())
connection.commit()

# Close the database connection
connection.close()